![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/VOICE_OF_THE_PATIENTS.ipynb)

# **Voice of the Patients-NER**

📌To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.3.1
Spark NLP_JSL Version : 4.3.1


## **`ner_vop_slim_wip`**

In [4]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

clinical_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained("ner_vop_slim_wip", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter   
    ])

sample_texts = ["Hi, Please I need your help! Two months ago I tested positive for H. Pylori however I have never experienced any pain or stomach discomfort in my life. My doctor (I am from an easter european country) insisted that I do an endoscopy. I was convinced that it is a very safe procedure and I agreed. However the same day after the upper endoscopy was performed I started to experience stomach burning, pain, and heartburn / chest pain. So bad that I had to go to the emergency. Endoscopy showed a chronic gastritis and duodenitis. However, my GI doctor did not think I need antibiotics since the gastritis was mild. I was taking PPI for 3-4 weeks but my symptoms were getting worse with no improvement, stomach pain was getting stabby and unbearable and it started to hurt a lot on the right side under the ribs. It also feels like something is swollen under right ribs and makes popping sound (ultrasound did not show anything). After PPI was not helping much I was put on Sucralfate (also called Carafate) and it helped a bit. Now it is been 2 months, I am back in Canada where I am residing permanently and doctors here have no idea what is going on. They sent me to redo the H.Pylori test and another theory they have is a bile reflux. I am really scared because for over 2 months I have not had a day without pain even on the better days, I eat very special diet and I lost some weight because I could not eat enough and underweight. I saw 3 GI doctors (one in Russia and 2 in Canada) and they all say that they have never seen a case like me... to get problems after endoscopy...",
"Ok, I have bad teeth. I went finally to the dentist and he said I have an infection in my bone under my bad tooth. I have been referred for removal of the decayed tooth. They prescribed me antibiotics 500mg of amoxicillin three times a day but I can't sleep, eat, or really think. I am taking ibroprofen and cocodamol regularly. No effect really, I am using clove oil, not that it helps. Anyway I just need to know if this pain will stop anytime soon? I now have swelling on the inside of my mouth in the roof of my mouth, it hurts to touch. Should I go to a and e? Is there anything I can do? My dentist is not open till Monday now. He said the antibiotics should work over the weekend but I am scared I won't make it through the weekend",
"Hi, Firstly I'd be grateful for any input from anyone, and understand that it won't be actual medical advice but I'm just looking at other options. Secondly, please don't pass me off simply telling me to go to a GP as many of you will be aware it's not that easy and certainly not for me. For around 2 years now I have been feeling more and more exhausting, tired, no matter how much I sleep, I never wake and feel like I've been asleep. The sleep is so intense even after say 12 hours of it I wake and feel utter rubbish, drained, and like I have been drinking all night, which I haven't. This has got so bad now I struggle to even see the day through without napping and still it's not enough. I've tried iron tablets, various other vitamins, etc, and noticed no difference, even taking Furrous Sulphate. For around a year now though at different, random times it seems I cannot swallow. I try but my body won't let me, I try and its like the signal from my brain isn't reaching my throat to swallow. As well as this I'm constantly zoned out, someone tries to talk to me but I can't process it or follow the conversation. I do have BPD, Autism and some other issues but I've had those my whole life while these new symptoms seem to be progressing. I'm really worried, and I know I will have to see a GP but I just wanted to see if anyone had any in put good or bad",
"Hi, I am not sure if this is the correct section to cover all of this, but I have been going through a tuff time recently, as a whole and one of the stand-out things is I can't cope generally with things and just can't sleep very much only about 3 hours a night. I used to sleep really well, but what's changed is when getting to the point of dropping off to sleep it's like my body or brain is just trying to fight it all the time, like releasing adrenaline (like a charge through my body) or a massive jerk happens numerous times. I have been taking Nytol, but that only gets me about 2-3 hours before waking up, my whole system seems like in a high state of alert throughout the day. Would Antidepressants help me and restore some calm to my system? I have never taken them before, but I feel as if I need something now, as I can't continue like this. Thanks",
"I'm only 25 and was prescribed lisnopril for high blood pressure. I also suffer from frequent headaches and migraines. Now the lisnopril actually did help with my headaches but the side effects are just too much. (Difficulty speaking, not myself, weakness) I made the decision to stop taking them. It's been 7 days and I literally can barely talk without my voice squeaking and sounding retarded. Im like on the verge of losing my job fuxk these poison pills like wtf do they give out to people. "]


data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_vop_slim_wip download started this may take some time.
[OK!]


In [16]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+--------------+-----+---+------------+----------+
|chunk         |begin|end|ner_label   |confidence|
+--------------+-----+---+------------+----------+
|Two months ago|29   |42 |DateTime    |0.8388667 |
|H. Pylori     |66   |74 |Disease     |0.57053334|
|pain          |113  |116|Symptom     |0.9948    |
|stomach       |121  |127|BodyPart    |0.9129    |
|discomfort    |129  |138|Symptom     |0.957     |
|doctor        |155  |160|Employment  |0.9433    |
|endoscopy     |223  |231|Procedure   |0.9962    |
|same day after|309  |322|DateTime    |0.46706668|
|upper         |328  |332|Laterality  |0.9321    |
|endoscopy     |334  |342|Procedure   |0.9892    |
|stomach       |382  |388|BodyPart    |0.8552    |
|burning       |390  |396|Symptom     |0.9104    |
|pain          |399  |402|Symptom     |0.9971    |
|heartburn     |409  |417|Symptom     |0.9945    |
|chest         |421  |425|BodyPart    |0.9506    |
|pain          |427  |430|Symptom     |0.9762    |
|emergency     |464  |472|Clini

In [6]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)
